## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
from random import random

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Options,
    Session,
    Sampler,
    RuntimeOptions,
)

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
num_qubits = 8
circuit = EfficientSU2(
    num_qubits=num_qubits,
    reps=2,
    entanglement="linear",
    su2_gates=['ry'],
    insert_barriers=False,
)

circuit = circuit.decompose()

params = [random() for _ in range(len(circuit.parameters))] # define list of parameter values
circuit = circuit.bind_parameters(params) # bind those values
circuit.draw(fold=300)

┌──────────────────────┐     ┌───────────────────────┐                                                   ┌───────────────────────┐                                                                                                                                                        
q_0: ─┤ Ry(0.65198909000809) ├──■──┤ Ry(0.988234695503339) ├──────────────────────────────────────■────────────┤ Ry(0.700210056630551) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     ┌┴──────────────────────┤┌─┴─┐└───────────────────────┘┌────────────────────────┐          ┌─┴─┐          └───────────────────────┘ ┌───────────────────────┐                                                                                                                              
q_1: ┤ Ry(0.369564357699531) ├┤ X ├────────────■────────────┤ Ry(0.0134183442989021) ├──────────┤ X ├──────────────────────■─────────────┤ Ry(0.778842281176651) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     ├───────────────────────┤└───┘          ┌─┴─┐          └────────────────────────┘┌─────────┴───┴─────────┐          ┌─┴─┐           └───────────────────────┘┌───────────────────────┐                                                                                                     
q_2: ┤ Ry(0.714416341793536) ├───────────────┤ X ├──────────────────────■─────────────┤ Ry(0.431518905868251) ├──────────┤ X ├───────────────────────■────────────┤ Ry(0.351554156328087) ├─────────────────────────────────────────────────────────────────────────────────────────────────────
     ├───────────────────────┤               └───┘                    ┌─┴─┐           └───────────────────────┘┌─────────┴───┴──────────┐          ┌─┴─┐          └───────────────────────┘┌───────────────────────┐                                                                            
q_3: ┤ Ry(0.256619154075708) ├────────────────────────────────────────┤ X ├───────────────────────■────────────┤ Ry(0.0652281641222194) ├──────────┤ X ├──────────────────────■────────────┤ Ry(0.803713264696147) ├────────────────────────────────────────────────────────────────────────────
     ├───────────────────────┤                                        └───┘                     ┌─┴─┐          └────────────────────────┘┌─────────┴───┴─────────┐          ┌─┴─┐          └───────────────────────┘┌────────────────────────┐                                                  
q_4: ┤ Ry(0.844212543172973) ├──────────────────────────────────────────────────────────────────┤ X ├──────────────────────■─────────────┤ Ry(0.997429147686717) ├──────────┤ X ├──────────────────────■────────────┤ Ry(0.0702973807985325) ├──────────────────────────────────────────────────
     ├───────────────────────┤                                                                  └───┘                    ┌─┴─┐           └───────────────────────┘ ┌────────┴───┴─────────┐          ┌─┴─┐          └────────────────────────┘┌───────────────────────┐                         
q_5: ┤ Ry(0.856460349529891) ├───────────────────────────────────────────────────────────────────────────────────────────┤ X ├───────────────────────■─────────────┤ Ry(0.66261244230681) ├──────────┤ X ├──────────────────────■─────────────┤ Ry(0.894280783588127) ├─────────────────────────
     ├───────────────────────┤                                                                                           └───┘                     ┌─┴─┐           └──────────────────────┘┌─────────┴───┴─────────┐          ┌─┴─┐           └───────────────────────┘┌───────────────────────┐
q_6: ┤ Ry(0.579804569917784) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├──────────────────────■────────────┤ Ry(0.081262581407529) ├──────────┤ X ├──────────────────────

### Set up the Qiskit runtime service

In [3]:
service = QiskitRuntimeService(
    channel="ibm_quantum",
    token="<YOUR_API_KEY>",
)

### Find cuts that match our criteria

In [4]:
# Set the Sampler and runtime options
options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})

# Run 2 parallel qasm simulator threads
backend_names = ["ibmq_qasm_simulator"] * 2

# Instantiate a WireCutter and decompose the circuit
cutter = WireCutter(
    circuit, service=service, backend_names=backend_names, options=options
)
# cutter = WireCutter(circuit) # local Estimator

cuts = cutter.decompose(
    method="automatic",
    max_subcircuit_width=6,
    max_cuts=2,
    num_subcircuits=[2],
)

2022-10-11 17:57:16,107	INFO worker.py:1518 -- Started a local Ray instance.


(_cut_automatic pid=15944) Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
(_cut_automatic pid=15944) Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
(_cut_automatic pid=15944) CPXPARAM_Read_DataCheck                          1
(_cut_automatic pid=15944) CPXPARAM_TimeLimit                               300
(_cut_automatic pid=15944) Warning:  Non-integral bounds for integer variables rounded.
(_cut_automatic pid=15944) Tried aggregator 3 times.
(_cut_automatic pid=15944) MIP Presolve eliminated 37 rows and 8 columns.
(_cut_automatic pid=15944) MIP Presolve modified 7 coefficients.
(_cut_automatic pid=15944) Aggregator did 103 substitutions.
(_cut_automatic pid=15944) Reduced MIP has 366 rows, 127 columns, and 1072 nonzeros.
(_cut_automatic pid=15944) Reduced MIP has 121 binaries, 6 generals, 0 SOSs, and 0 indicators.
(_cut_automatic pid=15944) Presolve time = 0.00 sec. (2.11 ticks)
(_cut_automatic pid=15944) Found incumbent of value 2.0

### Evaluate the subcircuits, then recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [5]:
# Evaluate the subcircuits on backend
subcircuit_instance_probabilities = cutter.evaluate(cuts)

# Recompose the circuit and generate the cut circuit's probability distribution
reconstructed_probabilities = cutter.recompose(
    subcircuit_instance_probabilities, cuts, num_threads=4
)

# Use a statevector simulator to calculate the error between the inferred and actual distributions
metrics = cutter.verify(reconstructed_probabilities)
print(metrics)

{'nearest': {'chi2': 0.016066288906780395, 'Mean Squared Error': 2.62806537740421e-07, 'Mean Absolute Percentage Error': 3868.8107416484286, 'Cross Entropy': 4.4287363434327744, 'HOP': 0.9734068302603233}, 'naive': {'chi2': 0.015953589469409044, 'Mean Squared Error': 2.5964347386122023e-07, 'Mean Absolute Percentage Error': 4145.918916629806, 'Cross Entropy': 4.415785860637712, 'HOP': 0.9706102949223668}}
